In [1]:
# It is one time installation
# !pip install boto3

In [2]:
# importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
import boto3
s3 = boto3.client('s3')

In [4]:
bucket_name = 's3bucketshalini'
object_key = 'heart.csv'
file_name = 'heart.csv'

s3.download_file(bucket_name, object_key, file_name)

In [5]:
dataset = pd.read_csv(file_name)

In [6]:
dataset.shape

(1025, 14)

In [7]:
dataset.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


In [9]:
dataset.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,1025.000000,1025.000000,1025.000000,1025.000000,1025.00000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000
mean,54.434146,0.695610,0.942439,131.611707,246.00000,0.149268,0.529756,149.114146,0.336585,1.071512,1.385366,0.754146,2.323902,0.513171
std,9.072290,0.460373,1.029641,17.516718,51.59251,0.356527,0.527878,23.005724,0.472772,1.175053,0.617755,1.030798,0.620660,0.500070
min,29.000000,0.000000,0.000000,94.000000,126.00000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,48.000000,0.000000,0.000000,120.000000,211.00000,0.000000,0.000000,132.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,56.000000,1.000000,1.000000,130.000000,240.00000,0.000000,1.000000,152.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,275.00000,0.000000,1.000000,166.000000,1.000000,1.800000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.00000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [10]:
dataset.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [11]:
dataset['target'].value_counts()

target
1    526
0    499
Name: count, dtype: int64

In [12]:
continuos_cols = []
for column in dataset.columns:
    if len(dataset[column].unique()) < 10:
        print(f"Value counts for {column}:\n{dataset[column].value_counts()}\n")
    else:
        continuos_cols.append(column)
        print(f"{column} has too many unique values to use value_counts.\n")
    print('-----------------------------------------------------------------')

age has too many unique values to use value_counts.

-----------------------------------------------------------------
Value counts for sex:
sex
1    713
0    312
Name: count, dtype: int64

-----------------------------------------------------------------
Value counts for cp:
cp
0    497
2    284
1    167
3     77
Name: count, dtype: int64

-----------------------------------------------------------------
trestbps has too many unique values to use value_counts.

-----------------------------------------------------------------
chol has too many unique values to use value_counts.

-----------------------------------------------------------------
Value counts for fbs:
fbs
0    872
1    153
Name: count, dtype: int64

-----------------------------------------------------------------
Value counts for restecg:
restecg
1    513
0    497
2     15
Name: count, dtype: int64

-----------------------------------------------------------------
thalach has too many unique values to use value_counts.


In [13]:
continuos_cols

['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

In [14]:
def outlier(col):
    Q1 = dataset[col].quantile(0.25)
    Q3 = dataset[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR

    upper_array = np.where(dataset[col] >= upper)[0]
    lower_array = np.where(dataset[col] <= lower)[0]

    dataset.drop(index = upper_array, inplace = True)
    dataset.drop(index = lower_array, inplace = True)
for col in continuos_cols:
    if '[609] not found in axis':
        continue
    else:
         outlier(col)

In [15]:
dataset.shape

(1025, 14)

In [16]:
dataset.corr()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
age,1.000000,-0.103240,-0.071966,0.271121,0.219823,0.121243,-0.132696,-0.390227,0.088163,0.208137,-0.169105,0.271551,0.072297,-0.229324
sex,-0.103240,1.000000,-0.041119,-0.078974,-0.198258,0.027200,-0.055117,-0.049365,0.139157,0.084687,-0.026666,0.111729,0.198424,-0.279501
cp,-0.071966,-0.041119,1.000000,0.038177,-0.081641,0.079294,0.043581,0.306839,-0.401513,-0.174733,0.131633,-0.176206,-0.163341,0.434854
trestbps,0.271121,-0.078974,0.038177,1.000000,0.127977,0.181767,-0.123794,-0.039264,0.061197,0.187434,-0.120445,0.104554,0.059276,-0.138772
chol,0.219823,-0.198258,-0.081641,0.127977,1.000000,0.026917,-0.147410,-0.021772,0.067382,0.064880,-0.014248,0.074259,0.100244,-0.099966
fbs,0.121243,0.027200,0.079294,0.181767,0.026917,1.000000,-0.104051,-0.008866,0.049261,0.010859,-0.061902,0.137156,-0.042177,-0.041164
restecg,-0.132696,-0.055117,0.043581,-0.123794,-0.147410,-0.104051,1.000000,0.048411,-0.065606,-0.050114,0.086086,-0.078072,-0.020504,0.134468
thalach,-0.390227,-0.049365,0.306839,-0.039264,-0.021772,-0.008866,0.048411,1.000000,-0.380281,-0.349796,0.395308,-0.207888,-0.098068,0.422895
exang,0.088163,0.139157,-0.401513,0.061197,0.067382,0.049261,-0.065606,-0.380281,1.000000,0.310844,-0.267335,0.107849,0.197201,-0.438029
oldpeak,0.208137,0.084687,-0.174733,0.187434,0.064880,0.010859,-0.050114,-0.349796,0.310844,1.000000,-0.575189,0.221816,0.202672,-0.438441


not needed for model till above

In [17]:
correlation_matrix = dataset.corr()
correlations_with_target = correlation_matrix['target'].abs().sort_values(ascending = False)
top_features = correlations_with_target[1:11].index.tolist()

In [18]:
top_features

['oldpeak',
 'exang',
 'cp',
 'thalach',
 'ca',
 'slope',
 'thal',
 'sex',
 'age',
 'trestbps']

In [19]:
top_5 = top_features[:5]
top_5

['oldpeak', 'exang', 'cp', 'thalach', 'ca']

In [20]:
selected_features = top_5 + ['target']
selected_data = dataset[selected_features]

In [21]:
selected_data.shape

(1025, 6)

In [22]:
selected_data.head(2)

,oldpeak,exang,cp,thalach,ca,target
0,1.0,0,0,168,2,0
1,3.1,1,0,155,0,0


In [23]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(selected_data, test_size = 0.3, random_state = 42)

In [24]:
train_data, validation_data = train_test_split(train_data, test_size = 0.2, random_state = 42)

In [25]:
print(train_data.shape, test_data.shape, validation_data.shape)

(573, 6) (308, 6) (144, 6)


In [26]:
train_data = train_data[['target'] + [col for col in train_data.columns if col != 'target']]
validation_data = validation_data[['target'] + [col for col in validation_data.columns if col != 'target']]
test_data = test_data[['target'] + [col for col in test_data.columns if col != 'target']]

In [27]:
# Convert DataFrames to CSV
train_data.to_csv('train.csv', index=False)
validation_data.to_csv('validation.csv', index=False)
test_data.to_csv('test.csv', index=False)

In [28]:
import sagemaker

# Initialize a sagemaker.Session object
sagemaker_session = sagemaker.Session()

# Define your bucket name
bucket = sagemaker_session.default_bucket()
prefix = ''
# Upload the data to S3 (stored at the root of the bucket)
s3_train_path = sagemaker_session.upload_data('train.csv', bucket=bucket, key_prefix = prefix)
s3_val_path = sagemaker_session.upload_data('validation.csv', bucket=bucket, key_prefix = prefix)
s3_test_path = sagemaker_session.upload_data('test.csv', bucket=bucket, key_prefix = prefix)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [29]:
s3_train_path

's3://sagemaker-us-east-2-274082713924//train.csv'

XGBoost Model

In [30]:
import boto3
from io import StringIO
def read_csv_from_s3(s3_path, header_present=True):
    """Read a CSV file from S3 into a DataFrame."""
    s3_client = boto3.client('s3')
    bucket, key = s3_path.replace("s3://", "").split("/", 1)
    csv_obj = s3_client.get_object(Bucket=bucket, Key=key)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')

    header = 0 if header_present else None
    df = pd.read_csv(StringIO(csv_string), header=header)
    return df

def write_csv_to_s3(df, s3_path):
    """Write a DataFrame to a CSV file in S3."""
    s3_client = boto3.client('s3')
    bucket, key = s3_path.replace("s3://", "").split("/", 1)
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index = False, header = False)

    s3_client.put_object(Bucket = bucket, Key = key, Body = csv_buffer.getvalue())


In [31]:
# Remove headers from the training dataset
df_train = read_csv_from_s3(s3_train_path, header_present=True)
write_csv_to_s3(df_train, s3_train_path)  # This will overwrite the original file

# Remove headers from the validation dataset
df_val = read_csv_from_s3(s3_val_path, header_present=True)
write_csv_to_s3(df_val, s3_val_path)  # This will overwrite the original file

# Remove headers from the validation dataset
df_test = read_csv_from_s3(s3_test_path, header_present=True)
write_csv_to_s3(df_test, s3_test_path)  # This will overwrite the original file

In [32]:
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker.image_uris import retrieve

role = get_execution_role()
sagemaker_session = sagemaker.Session()

# Get the URI for the XGBoost container
xgboost_container = retrieve('xgboost', sagemaker_session.boto_region_name, '1.3-1')

# SageMaker XGBoost model
xgb_estimator = sagemaker.estimator.Estimator(
    image_uri=xgboost_container,
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    hyperparameters={
        'objective': 'binary:logistic',
        'num_round': 100,
        'max_depth': 5,
        'eta': 0.2,
        'min_child_weight': 5,
        'subsample': 0.5
    }
)
xgb_estimator.fit({'train': TrainingInput(s3_train_path, content_type='csv'), 
                   'validation': TrainingInput(s3_val_path, content_type='csv')})


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-12-05-02-04-48-252


2023-12-05 02:04:48 Starting - Starting the training job...
2023-12-05 02:05:13 Starting - Preparing the instances for training.........
2023-12-05 02:06:31 Downloading - Downloading input data...
2023-12-05 02:07:01 Training - Downloading the training image...
2023-12-05 02:07:46 Training - Training image download completed. Training in progress...[2023-12-05 02:07:56.892 ip-10-0-205-155.us-east-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-12-05 02:07:56.921 ip-10-0-205-155.us-east-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2023-12-05:02:07:56:INFO] Imported framework sagemaker_xgboost_container.training
[2023-12-05:02:07:56:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2023-12-05:02:07:56:INFO] No GPUs detected (normal if no gpus installed)
[2023-12-05:02:07:56:INFO] Running XGBoost Sagemaker in algorithm mode
[2023-12-05:02:07:56:INFO] Determined

In [ ]:
# As I was working on building models, many models were deployed. But there there a limit on number of 
# instances which is 4. so I used below code get existing endpoints and delete it
'''
import boto3

sagemaker_client = boto3.client('sagemaker')
response = sagemaker_client.list_endpoints()

for endpoint in response['Endpoints']:
    print(f"Endpoint name: {endpoint['EndpointName']} - Status: {endpoint['EndpointStatus']}")
'''

In [ ]:
'''
import boto3
client = boto3.client('sagemaker')
client.delete_endpoint(EndpointName='sagemaker-xgboost-2023-12-05-01-22-19-950')
'''

In [33]:
xgb_predictor = xgb_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-12-05-02-08-30-615
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2023-12-05-02-08-30-615
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2023-12-05-02-08-30-615


-----!

In [34]:
import boto3
import pandas as pd
from io import StringIO

def read_csv_from_s3(s3_path, header_present=False):
    """Read a CSV file from S3 into a DataFrame."""
    s3_client = boto3.client('s3')
    bucket, key = s3_path.replace("s3://", "").split("/", 1)
    csv_obj = s3_client.get_object(Bucket=bucket, Key=key)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')

    header = 0 if header_present else None
    df = pd.read_csv(StringIO(csv_string), header=header)
    return df

# Load the test data
test_data = read_csv_from_s3(s3_test_path)

# Separate features
test_features = test_data.iloc[:, 1:]
test_labels = test_data.iloc[:, 0]

In [36]:
import io

# Convert the DataFrame to a CSV string
test_features_buffer = io.StringIO()
test_features.to_csv(test_features_buffer, header=False, index=False)

test_features_csv = test_features_buffer.getvalue()

In [38]:
runtime = boto3.client('sagemaker-runtime')

response = runtime.invoke_endpoint(
    EndpointName='sagemaker-xgboost-2023-12-05-02-08-30-615',  # Replace with your endpoint name
    ContentType='text/csv',
    Body=test_features_csv
)

# Retrieve prediction results
predictions = response['Body'].read().decode('utf-8')

In [39]:
predictions

'0.9177568554878235\n0.8187090754508972\n0.018550219014286995\n0.955284595489502\n0.045836351811885834\n0.6219742894172668\n0.010305017232894897\n0.006182427983731031\n0.8219638466835022\n0.0400492362678051\n0.7148256301879883\n0.011215360835194588\n0.9741403460502625\n0.8005146384239197\n0.010305017232894897\n0.5189552903175354\n0.0020618641283363104\n0.9558919072151184\n0.9559027552604675\n0.0075119961984455585\n0.06776176393032074\n0.02155863307416439\n0.5042837858200073\n0.009932253509759903\n0.06776176393032074\n0.841632068157196\n0.7355148196220398\n0.8176082968711853\n0.06621259450912476\n0.8927596211433411\n0.13464951515197754\n0.662796676158905\n0.7098833918571472\n0.8187090754508972\n0.893113911151886\n0.9675484299659729\n0.06776176393032074\n0.9605341553688049\n0.6662798523902893\n0.9683886766433716\n0.006182427983731031\n0.06962742656469345\n0.8136203289031982\n0.5042837858200073\n0.006519922520965338\n0.0400492362678051\n0.09571636468172073\n0.33355891704559326\n0.89311391

In [40]:
probability_scores = predictions.split('\n')
threshold = 0.5
predicted_labels = [1 if float(score) > threshold else 0 for score in probability_scores if score.strip()]


In [41]:
len(predicted_labels)

308

In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def metrics(test_labels, predictions):
    # Calculate metrics
    accuracy = accuracy_score(test_labels, predictions)
    precision = precision_score(test_labels, predictions)
    recall = recall_score(test_labels, predictions)
    f1 = f1_score(test_labels, predictions)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
metrics(test_labels, predicted_labels)

Accuracy: 0.8571428571428571
Precision: 0.8260869565217391
Recall: 0.8926174496644296
F1 Score: 0.8580645161290322


In [43]:
from sagemaker import LinearLearner, image_uris
from sagemaker.inputs import TrainingInput
from sagemaker.estimator import Estimator

# Define the algorithm name
algorithm_name = 'linear-learner'

# Get the current AWS region
region_name = sagemaker.Session().boto_region_name

# Retrieve the image URI for Linear Learner
image_uri = image_uris.retrieve(framework=algorithm_name, region=region_name, version='latest')

# Set up the SageMaker Estimator with the retrieved image URI
linear_learner_estimator = Estimator(image_uri=image_uri,
                                     role=sagemaker.get_execution_role(),
                                     instance_count=1,
                                     instance_type='ml.m4.xlarge',
                                     sagemaker_session=sagemaker.Session(),
                                     hyperparameters={
                                         'predictor_type': 'binary_classifier',
                                         'mini_batch_size': 100,
                                         'epochs': 20,
                                         'learning_rate': 0.02,
                                         'use_bias': True
                                     })

linear_learner_estimator.fit({'train': TrainingInput(s3_train_path, content_type='text/csv'), 
                   'validation': TrainingInput(s3_val_path, content_type='text/csv')})

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


INFO:sagemaker:Creating training-job with name: linear-learner-2023-12-05-02-12-53-815


2023-12-05 02:12:54 Starting - Starting the training job...
2023-12-05 02:13:18 Starting - Preparing the instances for training.........
2023-12-05 02:14:28 Downloading - Downloading input data...
2023-12-05 02:14:58 Training - Downloading the training image......
2023-12-05 02:16:19 Training - Training image download completed. Training in progress....Docker entrypoint called with argument(s): train
Running default environment configuration script
[12/05/2023 02:16:40 INFO 140643886823232] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'los

In [44]:
# deploy model
linear_learner_predictor = linear_learner_estimator.deploy(initial_instance_count=1, instance_type = 'ml.m4.xlarge')

INFO:sagemaker:Creating model with name: linear-learner-2023-12-05-02-17-37-322
INFO:sagemaker:Creating endpoint-config with name linear-learner-2023-12-05-02-17-37-322
INFO:sagemaker:Creating endpoint with name linear-learner-2023-12-05-02-17-37-322


-------!

In [45]:
runtime = boto3.client('sagemaker-runtime')

response = runtime.invoke_endpoint(
    EndpointName='linear-learner-2023-12-05-02-17-37-322',  # Replace with your endpoint name
    ContentType='text/csv',
    Body=test_features_csv
)

# Retrieve prediction results
predictions = response['Body'].read().decode('utf-8')

In [46]:
import json
# Parse the JSON response
parsed_predictions = json.loads(predictions)

# Extract only the predicted labels
predicted_labels = [prediction['predicted_label'] for prediction in parsed_predictions['predictions']]
print(predicted_labels)


[1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1]


In [47]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def metrics(test_labels, predictions):
    # Calculate metrics
    accuracy = accuracy_score(test_labels, predictions)
    precision = precision_score(test_labels, predictions)
    recall = recall_score(test_labels, predictions)
    f1 = f1_score(test_labels, predictions)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
metrics(test_labels, predicted_labels)

Accuracy: 0.814935064935065
Precision: 0.7705882352941177
Recall: 0.8791946308724832
F1 Score: 0.8213166144200628


In [48]:
xgb_predictor.delete_endpoint()
linear_learner_predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: sagemaker-xgboost-2023-12-05-02-08-30-615
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2023-12-05-02-08-30-615
INFO:sagemaker:Deleting endpoint configuration with name: linear-learner-2023-12-05-02-17-37-322
INFO:sagemaker:Deleting endpoint with name: linear-learner-2023-12-05-02-17-37-322


In [ ]:
'''
import boto3

sagemaker_client = boto3.client('sagemaker')
response = sagemaker_client.list_endpoints()

for endpoint in response['Endpoints']:
    print(f"Endpoint name: {endpoint['EndpointName']} - Status: {endpoint['EndpointStatus']}")
'''

In [ ]:
'''
import boto3
client = boto3.client('sagemaker')
client.delete_endpoint(EndpointName='sagemaker-xgboost-2023-12-05-01-22-19-950')
'''

Results :
My XGBoost model resulted in 85.7% while Linear learner model resulted in 81.4 accuracy. Since XGBoost model performed better than linear learner model, I used XGBoost model to predict results based on inputs from user. I deleted enpoints after taking all the screenshots.